In [1]:
import time
from pathlib import Path

import roach

from relbench.base import TaskType
from relbench.datasets import get_dataset_names
from relbench.tasks import get_task, get_task_names

In [2]:
project = "relbench/2024-07-05"
queue_gpu = "relbench_gpu"
queue_cpu = "relbench_cpu"

In [3]:
task_files = {}

In [4]:
def join(task_keys):
    tests = []
    for task_key in task_keys:
        task_file = task_files[task_key]
        task_file = task_file.replace("/ready/", "/done/")
        test = f"test -f {task_file}"
        tests.append(test)
    test = " && ".join(tests)
    return test

# download and cache materialized tensorframes

In [5]:
# clear current cache with backup
ts = int(time.time())
home = Path.home()
try:
    Path(f"{home}/.cache/relbench").rename(
        f"{home}/.cache/relbench.{ts}"
    )
except FileNotFoundError:
    pass

In [6]:
# clear current cache with backup
ts = int(time.time())
home = Path.home()
try:
    Path(f"{home}/.cache/relbench_examples").rename(
        f"{home}/.cache/relbench_examples.{ts}"
    )
except FileNotFoundError:
    pass

In [7]:
for dataset in get_dataset_names():
    task = get_task_names(dataset)[0]
    task_obj = get_task(dataset, task)
    if task_obj.task_type.value == TaskType.LINK_PREDICTION.value:
        script = "gnn_link"
    else:
        script = "gnn_node"
    # epochs=0 throws exception
    cmd = f"python {script}.py --dataset {dataset} --task {task} --epochs 1"
    task_files[f"prep_dataset/{dataset}"] = roach.submit(queue_gpu, cmd)

In [8]:
# download all tasks beforehand
for dataset in get_dataset_names():
    for task in get_task_names(dataset):
        py_cmd = (
            f"from relbench.tasks import download_task; "
            f'download_task("{dataset}", "{task}")'
        )
        assert "'" not in py_cmd
        cmd = f"python -c '{py_cmd}'"
        task_files[f"prep_task/{dataset}/{task}"] = roach.submit(queue_cpu, cmd)

# training runs

In [9]:
for seed in range(5):
    for dataset in get_dataset_names():
        for task in get_task_names(dataset):
            task_obj = get_task(dataset, task)
            if task_obj.task_type.value == TaskType.LINK_PREDICTION.value:
                scripts = [
                    "gnn_link",
                    "idgnn_link",
                    "lightgbm_link",
                    "baseline_link",
                ]
            else:
                scripts = [
                    "gnn_node",
                    "lightgbm_node",
                    "hybrid_node",
                    "baseline_node",
                ]
            for script in scripts:
                cmd = (
                    f"OMP_NUM_THREADS=8"
                    f" python {script}.py"
                    f" --dataset {dataset}"
                    f" --task {task}"
                    f" --seed {seed}"
                    f" --roach_project {project}"
                )
                if dataset == "rel-trial":
                    if script == "gnn_node":
                        cmd += f" --epochs 20"
                        if task_obj.task_type.value in [
                            TaskType.BINARY_CLASSIFICATION.value,
                            TaskType.MULTICLASS_CLASSIFICATION.value,
                        ]:
                            cmd += f" --lr 0.0001"
                            cmd += f" --num_neighbors 64"
                            cmd += f" --aggr mean"
                        elif task_obj.task_type.value == TaskType.REGRESSION.value:
                            cmd += f" --lr 0.01"
                    elif script == "idgnn_link":
                        cmd += f" --num_layers 4"
                requires = join([f"prep_dataset/{dataset}", f"prep_task/{dataset}/{task}"])
                if script in ["gnn_link", "idgnn_link", "gnn_node", "hybrid_node"]:
                    queue = queue_gpu
                elif script in ["lightgbm_link", "lightgbm_node", "baseline_link", "baseline_node"]:
                    queue = queue_cpu
                roach.submit(queue, cmd, requires)